In [65]:
%matplotlib inline

import numpy as np

import matplotlib.pyplot as plt

#this is the minimizer, you don't have to use this one if you are familiar with some other minimizer.
from iminuit import Minuit
import copy
from skhep.math.vectors import LorentzVector

In [66]:
# data = np.load("./test_temp/parton_tthvv_10TeV.npy")
#load the data here, keep in mind of the format of your own data
data = np.loadtxt('/Users/morgancassidy/tthvv_sm_parton_10tev')

In [67]:
mtop = 173.
mW = 80.4
mH = 125.
#set some parameters here.
# you can choose what sigma should be, in principle, they should depend own the width of the mass distributions.
sigmaH = 5
sigmat = 5
sigmaw = 5
sigmapt = 100

In [68]:
def getMass(E,Px,Py,Pz):
    m2= E**2-Px**2-Py**2-Pz**2
    if m2<0:
        return 0
    else :
        return np.sqrt(m2)
    
def getMassall(particle):
    E = particle[:,0]
    Px = particle[:,1]
    Py = particle[:,2]
    Pz = particle[:,3]
    m2= E**2-Px**2-Py**2-Pz**2
    return np.sqrt(m2)

#I defined some functions here

def chisquare(par0, par1, par2):
    
#This is the function to be minimized, the three parameters are the three unknown neutrino momentums.
#here I define momentum of all particles.
    Nu1px = par0
    Nu1py = par1
    Nu1pz = par2
    Nu1E = np.sqrt(Nu1px**2+Nu1py**2+Nu1pz**2)
    

#This is not the best way to do it, adjust according to your need.
#here Lep1,bot1.... are global parameters I set outside of the function
    L1E = Lep1[0]
    L1px = Lep1[1]
    L1py = Lep1[2]
    L1pz = Lep1[3]
    
    b1E = bot1[0]
    b1px = bot1[1]
    b1py = bot1[2]
    b1pz = bot1[3]
    
    b2E = bot2[0]
    b2px = bot2[1]
    b2py = bot2[2]
    b2pz = bot2[3]
    
    b3E = bot3[0]
    b3px = bot3[1]
    b3py = bot3[2]
    b3pz = bot3[3]
    
    b4E = bot4[0]
    b4px = bot4[1]
    b4py = bot4[2]
    b4pz = bot4[3]
    
    j1E = jet1[0]
    j1px = jet1[1]
    j1py = jet1[2]
    j1pz = jet1[3]
    
    j2E = jet2[0]
    j2px = jet2[1]
    j2py = jet2[2]
    j2pz = jet2[3]
    
    v111E = Nu1E+L1E+b2E
    v111px = Nu1px+L1px+b2px
    v111py = Nu1py+L1py+b2py
    v111pz = Nu1pz+L1pz+b2pz
    
    pt1 = np.sqrt(v111px**2+v111py**2)
    
    v111m = getMass(v111E, v111px, v111py, v111pz)
    N111 = (v111m-mtop)**2/(sigmat**2)
    
    v222E = j1E+j2E+b1E
    v222px = j1px+j2px+b1px
    v222py = j1py+j2py+b1py
    v222pz = j1pz+j2pz+b1pz
    
    pt2 = np.sqrt(v222px**2+v222py**2)
    
    v222m = getMass(v222E, v222px, v222py, v222pz)
    N222 = (v222m-mtop)**2/(sigmat**2)
    
    v11E = Nu1E+L1E
    v11px = Nu1px+L1px
    v11py = Nu1py+L1py
    v11pz = Nu1pz+L1pz
    
    v11m = getMass(v11E, v11px, v11py, v11pz)
    N11 = (v11m-mW)**2/(sigmaw**2)

    Npt = (pt1-pt2)**2/(sigmapt**2)
    
    H11 = getMass(b3E+b4E, b3px+b4px, b3py+b4py, b3pz+b4pz)
    
    NH = (H11-mH)**2/(sigmaH**2)
    
    chi2 =N111+N222+N11+Npt+NH
    
    #five terms of chi-squared. each term is (predicted-true)^2/sigma^2
    #mt+mtbar+mw(leptonic)+|pt_t-pt_tbar|+mH
    #More accurate the mass is compare to the true value, the smaller chi squared value we got.
    
    return chi2

In [69]:
#Here are all the b quark combinations to form t, tbar and H. also not the best way to do it.
pairlist = [[0,1,2,3],[1,0,2,3],[0,2,1,3],[2,0,1,3],[0,3,1,2],[3,0,1,2],[1,2,0,3],[2,1,0,3],[1,3,0,2],[3,1,0,2],[2,3,0,1],[3,2,0,1]]

In [70]:
for i in range(data.shape[0]):
#I set the momentums here.
    Lep1= data[i,[24,25,26,27]]
    jet1 = data[i,[16,17,18,19]]
    jet2 = data[i,[20,21,22,23]]
    
    blist = [data[i,[0,1,2,3]],data[i,[4,5,6,7]],data[i,[8,9,10,11]],data[i,[12,13,14,15]]]
    
    sol = []
    p_nu_temp = []

#For each of the b quark combinations calculate the chi sqaured value
    for pair in pairlist:
        bot1= blist[pair[0]]
        bot2= blist[pair[1]]
        bot3= blist[pair[2]]
        bot4= blist[pair[3]]

In [71]:
print(bot1)
print(bot2)
print(bot3)
print(bot4)

[ 175.06419592  -40.99197736  -66.55953405 -156.57224822]
[ 1472.9793663   -669.27117968  -600.25177226 -1166.7990494 ]
[ 255.09862229 -136.73271374 -204.04158158  -68.7343806 ]
[ 170.36300478 -143.36415621  -90.90413235   13.58752489]


In [72]:
print(data.shape)
print(Lep1.shape)
print(data[0])
Lep1
jet1
jet2

(4428, 32)
(4,)
[  248.43458173    71.57010046  -117.47830155   206.819295
   419.27340239  -137.4083114    127.94034137   374.85773429
    31.79452249     3.33771628    14.69128395   -27.60122255
  2603.097769     115.61192101   206.79771518 -2592.2894309
    24.08218524    14.86179435     7.62092237    17.34935898
   122.70502828   -37.82613556   -96.56267017    65.58474031
   329.78250549  -168.87500807    99.23007657   265.31325726
    83.02015056   -71.95092306    14.9598078     38.62142178]


array([ 334.16136353, -190.90720683, -273.83842794,   15.19113704])

In [34]:
print(bot1)
print(data[0])
print(jet1)
print(jet2)
print(len(Lep1))

print(data[0,[0]])
print(data[0])
b1_e, b1_px, b1_py, b1_pz = 0, 1, 2, 3
b2_e, b2_px, b2_py, b2_pz = 4, 5, 6, 7
b3_e, b3_px, b3_py, b3_pz = 8, 9, 10, 11
b4_e, b4_px, b4_py, b4_pz = 12, 13, 14, 15
j1_e, j1_px, j1_py, j1_pz = 16, 17, 18, 19
j2_e, j2_px, j2_py, j2_pz = 20, 21, 22, 23
l1_e, l1_px, l1_py, l1_pz = 24, 25, 26, 27

[ 175.06419592  -40.99197736  -66.55953405 -156.57224822]
[  248.43458173    71.57010046  -117.47830155   206.819295
   419.27340239  -137.4083114    127.94034137   374.85773429
    31.79452249     3.33771628    14.69128395   -27.60122255
  2603.097769     115.61192101   206.79771518 -2592.2894309
    24.08218524    14.86179435     7.62092237    17.34935898
   122.70502828   -37.82613556   -96.56267017    65.58474031
   329.78250549  -168.87500807    99.23007657   265.31325726
    83.02015056   -71.95092306    14.9598078     38.62142178]
[ 1237.0062356   -629.01252995 -1062.4348107    -75.89424935]
[ 334.16136353 -190.90720683 -273.83842794   15.19113704]
4
[248.43458173]
[  248.43458173    71.57010046  -117.47830155   206.819295
   419.27340239  -137.4083114    127.94034137   374.85773429
    31.79452249     3.33771628    14.69128395   -27.60122255
  2603.097769     115.61192101   206.79771518 -2592.2894309
    24.08218524    14.86179435     7.62092237    17.34935898
   122.70502828  

In [73]:
%%capture
p_nu = []
index = []
for i in range(data.shape[0]):
#I set the momentums here.
    Lep1= data[i,[24,25,26,27]]
    jet1 = data[i,[16,17,18,19]]
    jet2 = data[i,[20,21,22,23]]
    
    blist = [data[i,[0,1,2,3]],data[i,[4,5,6,7]],data[i,[8,9,10,11]],data[i,[12,13,14,15]]]
    
    sol = []
    p_nu_temp = []

#For each of the b quark combinations calculate the chi sqaured value
    for pair in pairlist:
        bot1= blist[pair[0]]
        bot2= blist[pair[1]]
        bot3= blist[pair[2]]
        bot4= blist[pair[3]]
        chisquare.errordef = Minuit.LEAST_SQUARES
        m = Minuit(chisquare, par0 = 0, par1 = 0, par2 = 0)
        m.limits["par0", "par1", "par2"] = (-5000,5000)
        m.migrad()
        sol.append(m.fval)
        p_nu_temp.append(m.values)
#pick out the smallest chi sqaured value and save that solution of neutrino momemtum.
    I = np.argmin(sol)
    index.append(I)
    p_nu.append(p_nu_temp[I])
p_nu = np.array(p_nu)

In [ ]:
# Here is where I make mass distribution and save solutions.
# So that I can make histograms.
def get_momentum2(true, predict, index):
    mw = []
    mt = []
    mtbar = []
    mH = []
    pt_diff = []
    p_neu = []
    p_t = []
    for i in range(len(predict)):
        event = true[i]
        epredict = predict[i]
        I = index[i]
        blist = [event[[0,1,2,3]],event[[4,5,6,7]],event[[8,9,10,11]],event[[12,13,14,15]]]
        bot1= blist[pairlist[I][0]]
        bot2= blist[pairlist[I][1]]
        bot3= blist[pairlist[I][2]]
        bot4= blist[pairlist[I][3]]
        b1 = LorentzVector()
        l1 = LorentzVector()
        b2 = LorentzVector()
        b3 = LorentzVector()
        tbar = LorentzVector()
        neu1 = LorentzVector()
        b1.setpxpypze(bot2[1], bot2[2], bot2[3], bot2[0])
        l1.setpxpypze(event[25], event[26], event[27], event[24])
        neu1.setpxpypzm(epredict[0], epredict[1], epredict[2], 0)
        b2.setpxpypze(bot3[1], bot3[2], bot3[3], bot3[0])
        b3.setpxpypze(bot4[1], bot4[2], bot4[3], bot4[0])
        tbar.setpxpypze(bot1[1]+event[17]+event[21], bot1[2]+event[18]+event[22], bot1[3]+event[19]+event[23], bot1[0]+event[16]+event[20])
        mw.append((l1+neu1).m)
        mt.append((b1+l1+neu1).m)
        mtbar.append(tbar.m)
        pt_diff.append(np.abs((b1+l1+neu1).pt-tbar.pt))
        mH.append((b2+b3).m)
        p_neu.append(np.array([neu1.e,neu1.px,neu1.py,neu1.pz]))
        top = b1+l1+neu1
        p_t.append(np.array([top.e,top.px,top.py,top.pz]))
    return np.array(mw), np.array(mt), np.array(mtbar), np.array(pt_diff), np.array(mH), np.array(p_neu), np.array(p_t)


def save_momentums(true, predict, index):
    data = copy.deepcopy(true)
    for i in range(len(predict)):
        event = true[i]
        epredict = predict[i]
        I = index[i]
        blist = [event[[0,1,2,3]],event[[4,5,6,7]],event[[8,9,10,11]],event[[12,13,14,15]]]
        bot1= blist[pairlist[I][0]]
        bot2= blist[pairlist[I][1]]
        bot3= blist[pairlist[I][2]]
        bot4= blist[pairlist[I][3]]
        E = np.sqrt(epredict[0]**2+epredict[1]**2+epredict[2]**2)
        data[i,[28,29,30,31]] = np.array([E,epredict[0], epredict[1], epredict[2]])
        data[i,[0,1,2,3]] = bot1
        data[i,[4,5,6,7]] = bot2
        data[i,[8,9,10,11]] = bot3
        data[i,[12,13,14,15]] = bot4
    return data

In [ ]:
parton_mw, parton_mt, parton_mtbar, parton_tt_diff, parton_mh, parton_p_neu_predict, parton_p_t_predict  = get_momentum2(data, p_nu, index)

In [ ]:
parton = save_momentums(data, p_nu, index)
np.save('./test_temp/parton_tthvv_10TeV_chi2_fit.npy',parton)